In [0]:
import datetime

import numpy as np
import pandas as pd
import os

import io

In [0]:
EDUAW = pd.read_excel('dataset.xlsx', 'Education allowance', encoding='utf-8')

In [0]:
EDUAW = EDUAW[['Serial Number','Start date','End date','Wage Type']]

In [7]:
EDUAW.head()

,Serial Number,Start date,End date,Wage Type
0,446,2004-09-30,2004-09-30,Edu Assist'nce-Public
1,446,2006-06-26,2006-06-26,Edu Assist'nce-Public
2,446,2006-06-30,2006-06-30,Edu Assist'nce-Public
3,500,2005-04-30,2005-04-30,Edu Assist'nce-Public
4,500,2005-11-30,2005-11-30,Edu Assist'nce-Public


In [14]:
EDUAW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2063 entries, 0 to 2062
Data columns (total 4 columns):
Serial Number    2063 non-null int64
Start date       2063 non-null datetime64[ns]
End date         2063 non-null datetime64[ns]
Wage Type        2063 non-null object
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 64.5+ KB


In [0]:
#EDUAW.set_index('Serial Number')

In [10]:
EDUAW.head()

,Serial Number,Start date,End date,Wage Type
0,446,2004-09-30,2004-09-30,Edu Assist'nce-Public
1,446,2006-06-26,2006-06-26,Edu Assist'nce-Public
2,446,2006-06-30,2006-06-30,Edu Assist'nce-Public
3,500,2005-04-30,2005-04-30,Edu Assist'nce-Public
4,500,2005-11-30,2005-11-30,Edu Assist'nce-Public


In [26]:
now = datetime.datetime.today().year
now

2018

In [0]:
EDUAW['Start date Y'] = pd.DatetimeIndex(EDUAW['Start date']).year
EDUAW['End date Y'] = pd.DatetimeIndex(EDUAW['End date']).year
EDUAW['Present'] = now
EDUAW['Last Duration'] = now-EDUAW['End date Y']

In [45]:
EDUAW.head()

,Serial Number,Start date,End date,Wage Type,Start date Y,End date Y,Present,Last Duration
0,446,2004-09-30,2004-09-30,Edu Assist'nce-Public,2004,2004,2018,14
1,446,2006-06-26,2006-06-26,Edu Assist'nce-Public,2006,2006,2018,12
2,446,2006-06-30,2006-06-30,Edu Assist'nce-Public,2006,2006,2018,12
3,500,2005-04-30,2005-04-30,Edu Assist'nce-Public,2005,2005,2018,13
4,500,2005-11-30,2005-11-30,Edu Assist'nce-Public,2005,2005,2018,13


In [0]:
x = EDUAW.groupby(['Serial Number'])['Start date Y'].max().reset_index(name='MaxYear')

In [0]:
#EDUAWstat = EDUAW.groupby("Serial Number")

In [0]:
y = EDUAW.groupby(['Serial Number']).size().reset_index(name='All Counts')

In [0]:
z = EDUAW.groupby(['Serial Number','Start date Y']).size().reset_index(name='Year Counts')

In [133]:
EDUAW_df = x.set_index('Serial Number').join(y.set_index(['Serial Number']))
EDUAW_df.reset_index(inplace=True)
EDUAW_df.set_index(['Serial Number','MaxYear'], inplace=True)
EDUAW_df.head(5)

,,All Counts
Serial Number,MaxYear,
446,2006,3
500,2007,3
540,2013,1
543,2013,1
546,2014,1


In [134]:
z = z.rename({'Start date Y':'MaxYear'}, axis=1).set_index(['Serial Number','MaxYear'])
z.head(5)

Year Counts
Serial Number MaxYear             
446           2004               1
              2006               2
500           2005               2
              2007               1
540           2013               1

In [135]:
EDUAW_df = z.assign(AllCounts=EDUAW_df['All Counts']).dropna()
EDUAW_df.head()

,,Year Counts,AllCounts
Serial Number,MaxYear,,
446,2006,2,3.0
500,2007,1,3.0
540,2013,1,1.0
543,2013,1,1.0
546,2014,1,1.0


In [0]:
WORKF = pd.read_excel('dataset.xlsx', 'TH workforce', encoding='utf-8')[['Serial Number','Personnel Action Text','Personnel action - Year']]

In [72]:
WORKF.head()

,Serial Number,Personnel Action Text,Personnel action - Year
0,1,LEAVE THE COMPANY,1998
1,2,Withdrawn Chg. (Migration),1998
2,3,Withdrawn Chg. (Migration),1998
3,4,LEAVE THE COMPANY,2000
4,5,LEAVE THE COMPANY,2000


In [0]:
LEAVE_ACTIONS = [
    'LEAVE THE COMPANY',
    'Withdrawn Chg. (Migration)',
#     'RETIREMENT NOT PAID',
#     'INTERNAL MOVE',
#     'RETURN TO HOME COMPANY',
#     'CHANGE EMPLOYEE SITUATION',
#     'Hiring Active (Migration)',
#     'RE-ENTRY IN THE COMPANY',
#     'ENTRY IN THE COMPANY',
]
WORKF['Leave'] = WORKF['Personnel Action Text'].map({a:1 for a in LEAVE_ACTIONS}).fillna(0)
target = WORKF[['Serial Number', 'Leave', 'Personnel action - Year']].set_index('Serial Number')

df = target.join(EDUAW_df.reset_index().set_index('Serial Number'))
leave_df = df[df['Leave'] == 1.0].dropna(in)
not_leave_df = df[df['Leave'] != 1.0]



In [159]:
df[df.index == 446]

,Leave,Personnel action - Year,MaxYear,Year Counts,AllCounts,Last Duration
Serial Number,,,,,,
446,0.0,2014,2006.0,2.0,3.0,12.0


In [160]:
leave_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2820 entries, 1 to 4992
Data columns (total 6 columns):
Leave                      2820 non-null float64
Personnel action - Year    2820 non-null int64
MaxYear                    273 non-null float64
Year Counts                273 non-null float64
AllCounts                  273 non-null float64
Last Duration              273 non-null float64
dtypes: float64(5), int64(1)
memory usage: 234.2 KB


In [0]:
leave_df['Last Duration'] = leave_df['Personnel action - Year'] - leave_df['MaxYear']
now = datetime.datetime.today().year
leave_df[leave_df.index == 446]

In [154]:
now

2018

In [155]:
now = datetime.datetime.today().year
now

not_leave_df['Last Duration'] = now - not_leave_df['MaxYear']
not_leave_df.head()

not_leave_df[not_leave_df.index == 446]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Leave,Personnel action - Year,MaxYear,Year Counts,AllCounts,Last Duration
Serial Number,,,,,,
446,0.0,2014,2006.0,2.0,3.0,12.0


In [0]:
df = pd.concat([leave_df,not_leave_df])

In [157]:
df.head(5)

,Leave,Personnel action - Year,MaxYear,Year Counts,AllCounts,Last Duration
Serial Number,,,,,,
1,1.0,1998,NaN,NaN,NaN,NaN
2,1.0,1998,NaN,NaN,NaN,NaN
3,1.0,1998,NaN,NaN,NaN,NaN
4,1.0,2000,NaN,NaN,NaN,NaN
5,1.0,2000,NaN,NaN,NaN,NaN


In [158]:
df[df.index == 446]

,Leave,Personnel action - Year,MaxYear,Year Counts,AllCounts,Last Duration
Serial Number,,,,,,
446,0.0,2014,2006.0,2.0,3.0,12.0
